In [1]:
# imports
import pandas as pd

In [8]:
# Load data tables
df_symbol = pd.read_csv("../../seed/db_symbol_seed.csv")
df_holdings = pd.read_csv("../../seed/db_holdings_seed.csv")

In [9]:
df_symbol[df_symbol["symbol"] == "SPY"]

,id,symbol
512,513,SPY


In [43]:
# Create new table
df_hierarchy = pd.DataFrame(columns=["symbol_id", "type", "parent_symbol_id"])

In [44]:
market = ["SPY"]
sector = ["XLC", "XLY", "XLP", "XLE", "XLF", "XLV", "XLI", "XLB", "XLRE", "XLK", "XLU"]
# Business is everything else

In [45]:
# Add market to table
df_market_subset = df_symbol[df_symbol.symbol.isin(market)]
for i, r in df_market_subset.iterrows():
    row = pd.DataFrame({"symbol_id": [r.id], "type": ["MARKET"], "parent_symbol_id": [pd.NA]})
    df_hierarchy = pd.concat([df_hierarchy, row], ignore_index=True)

df_hierarchy.head()

,symbol_id,type,parent_symbol_id
0,513,MARKET,<NA>


In [46]:
# Add sectors to table
df_sector_subset = df_symbol[df_symbol.symbol.isin(sector)]
for i, r in df_sector_subset.iterrows():
    row = pd.DataFrame({"symbol_id": [r.id], "type": ["SECTOR"], "parent_symbol_id": [df_market_subset.id.iloc[0]]})
    df_hierarchy = pd.concat([df_hierarchy, row], ignore_index=True)

df_hierarchy.head()

,symbol_id,type,parent_symbol_id
0,513,MARKET,<NA>
1,492,SECTOR,513
2,493,SECTOR,513
3,494,SECTOR,513
4,495,SECTOR,513


In [48]:
# Add individual stocks to table
df_individual_subset = df_symbol[~df_symbol.symbol.isin(set(market+sector))]
for i, r in df_individual_subset.iterrows():
    parent_symbol_id = df_holdings[df_holdings.symbol_id == r.id]["holding_symbol_id"].iloc[0]
    row = pd.DataFrame({"symbol_id": [r.id], "type": ["INDIVIDUAL"], "parent_symbol_id": [parent_symbol_id]})
    df_hierarchy = pd.concat([df_hierarchy, row], ignore_index=True)
    
df_hierarchy

,symbol_id,type,parent_symbol_id
0,513,MARKET,<NA>
1,492,SECTOR,513
2,493,SECTOR,513
3,494,SECTOR,513
4,495,SECTOR,513
...,...,...,...
508,508,INDIVIDUAL,501
509,509,INDIVIDUAL,498
510,510,INDIVIDUAL,496
511,511,INDIVIDUAL,496


In [50]:
df_hierarchy.to_csv("../../seed/db_hierarchy_seed.csv", index=False)